In [1]:
import import_ipynb
import buildDescriptors as bd
import configparser



importing Jupyter notebook from buildDescriptors.ipynb
conf. file sections: ['flags', 'input files', 'EnergyBarrier', 'neural net']


# MachineLeranedMC

In [20]:
class MachineLeranedMC( bd.ParseConfiguration,
                        bd.EnergyBarrier,
                      ):
    '''
    Performs Machine Learned Monte Carlo Swaps
    '''
    
    def __init__(self,
                 confParser, 
                 verbose = False
                ):
        
        bd.ParseConfiguration.__init__(self, confParser, verbose = verbose )
        
        self.confParser  =  confParser
        
    def GetDescriptors( self ):
        '''
        Compute structural descriptors 
        '''
        
        bd.EnergyBarrier.__init__( self,
                                  '%s/EVENTS_DIR'%self.confParser['input files']['input_path'],
                                  '%s/EVLIST_DIR'%self.confParser['input files']['input_path'],
                                   self.lmpData,
                                   self.lmpDisp,
                                   verbose    = self.verbose,
                                   nconf      = 2, #--- only two events
                                   confParser = self.confParser,
                                   species    = ['Ni'], #'Ni Co Cr'.split()
                                   r_cut      = eval(self.confParser['EnergyBarrier']['r_cut']),
                                   dr         = eval(self.confParser['EnergyBarrier']['dr']),
                                   scale      = eval(self.confParser['EnergyBarrier']['scale']),
                                   n_max      = 8,
                                   l_max      = 6,

                      )
        
        self.perAtomData = self.lmpData
        self.SetDescriptors(
                      #soap = False,
                      #acsf = True,   
                      gr = True,
                     )
        
    def GetDefects( self ):
        '''
        Classify Defects
        '''
        
        #--- load ml model
        model = keras.models.load_model(fp) #"best_model/convnetClassifier_from_scratch.tf")

        #---------------
        #--- zscore X
        #---------------        
        X      = np.c_[self.descriptors ]
        scaler = StandardScaler()
        scaler.fit(X)
        X      = scaler.transform( X )

        #--- predict classes
        predict_x = model.predict( X ) 
        self.predict_classes = np.argmax( predict_x, axis=1 )
    
    
    def GetDisp( self ):
        '''
        Predict Displacements
        '''

        #--- load ml model
        model = keras.models.load_model(fp) #"best_model/convnetClassifier_from_scratch.tf")
        
        #---------------
        #--- zscore X
        #---------------        
        filtr  = self.predict_classes == 1
        X      = np.c_[self.descriptors[ filtr ] ]
        scaler = StandardScaler()
        scaler.fit(X)
        X      = scaler.transform( X )

        self.predict_disp = model.predict( X )
        
    
    def BuildCatalog( self ):
        rate_constant_prefactor = ?
        columns = 'AtomId barrier true_rate dx dy dz'.split()


## main()

In [21]:
#--- parse conf. file
confParser = configparser.ConfigParser()
confParser.read('configuration.ini')
print('conf. file sections:',confParser.sections())



mlmc  =  MachineLeranedMC(confParser,verbose = True)
!rm -r dumpFile; mkdir dumpFile
    
#--- parse allconf
mlmc.Parse('%s/%s'%(confParser['input files']['input_path'],confParser['input files']['dump_file']),
        'dumpFile/dump.xyz',
        )
    
#--- total displacements
!mkdir disp
mlmc.Displacement( 'dumpFile/dump.xyz',
                    'disp/disp.xyz'
                 )

#--- build descriptors
mlmc.GetDescriptors()

#--- identify defects
mlmc.GetDefects()

#--- predict diffusion paths
mlmc.GetDisp()


#--- build catalog
mlmc.BuildCatalog()

conf. file sections: ['flags', 'input files', 'EnergyBarrier', 'neural net']
output dump file=1.408613920211792 s
parsing ../simulations/niNatom1KTemp1000K3rd/Run0/allconf
elapsed time=0.0473628044128418 s
time steps: dict_keys([0, 1, 2])


,id,type,x,y,z
0,1,1,19.823929,18.054071,19.823696
1,2,1,0.353771,16.284138,2.124210
2,3,1,18.054001,16.284338,5.664271
3,4,1,3.893790,12.744229,16.283861
4,5,1,9.204130,5.663917,0.354000


mkdir: disp: File exists
output disp:1.3825290203094482 s
parsing disp/disp.xyz
elapsed time 0.06704401969909668 s


,id,type,x,y,z,DisplacementX,DisplacementY,DisplacementZ
0,1,1,19.823929,18.054071,19.823696,0.0,0.0,0.0
1,2,1,0.353771,16.284138,2.124210,0.0,0.0,0.0
2,3,1,18.054001,16.284338,5.664271,0.0,0.0,0.0
3,4,1,3.893790,12.744229,16.283861,0.0,0.0,0.0
4,5,1,9.204130,5.663917,0.354000,0.0,0.0,0.0


Build neighbor list: elapsed time=2.8871400356292725 s


,id,type,J,Jtype,DIST,DX,DY,DZ,PBC_SHIFT_X,PBC_SHIFT_Y,PBC_SHIFT_Z
0,1,1,1270.0,1.0,2.502422,1.769480,-1.769480,-0.000641,0.0,0.0,0.0
1,1,1,900.0,1.0,2.502919,1.769733,1.769928,0.000032,0.0,0.0,0.0
2,1,1,1340.0,1.0,2.503296,0.000023,1.769928,1.770267,0.0,0.0,0.0
3,1,1,1348.0,1.0,2.502764,-0.000206,1.769928,-1.769514,0.0,0.0,0.0
4,1,1,1099.0,1.0,2.503545,1.769184,0.001183,-1.771362,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
16435,1371,1,585.0,1.0,2.503169,1.770072,-0.000028,-1.769943,0.0,0.0,0.0
16436,1371,1,1055.0,1.0,2.503169,0.000028,-1.770073,-1.769943,0.0,0.0,0.0
16437,1371,1,275.0,1.0,2.503280,1.770044,1.770129,-0.000004,0.0,0.0,0.0
16438,1371,1,1100.0,1.0,2.503281,-1.770129,-1.770044,-0.000004,0.0,0.0,0.0


xdim= 21
ydim= 21
zdim= 21
kde: elapsed time=5.91464900970459 s
rho.shape:(1371,9261)


In [24]:
mlmc.descriptors

array([[1.36621930e-19, 1.60528039e-17, 2.82456009e-15, ...,
        1.75714630e-12, 1.49995887e-14, 1.90623576e-16],
       [1.35967458e-19, 1.60355702e-17, 2.82406503e-15, ...,
        1.75490775e-12, 1.49717095e-14, 1.90609413e-16],
       [1.34396746e-19, 1.58598701e-17, 2.79114211e-15, ...,
        1.75994958e-12, 1.50351142e-14, 1.90213878e-16],
       ...,
       [1.37384158e-19, 1.61441969e-17, 2.84431702e-15, ...,
        1.77214538e-12, 1.51161652e-14, 1.92545299e-16],
       [1.35808914e-19, 1.59907648e-17, 2.81922784e-15, ...,
        1.75088518e-12, 1.49237865e-14, 1.90663296e-16],
       [1.36070544e-19, 1.59994777e-17, 2.82062132e-15, ...,
        1.75260464e-12, 1.49392122e-14, 1.90906469e-16]])

In [25]:
dir(mlmc)

['DefectFreeCrystal',
 'Displacement',
 'GetCenterAtom',
 'GetDescriptors',
 'GetEventid',
 'LabelDefects',
 'LabelDefectsUpdated',
 'Parse',
 'ParseDiffusionPath',
 'ParseEvList_dir',
 'ParseEvents_dir',
 'Print',
 'PrintDescriptors',
 'PrintPerAtomData',
 'ReadConfigurationFile',
 'SetDescriptors',
 'SplitByType',
 'WignerSeitz',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'box',
 'confParser',
 'datFile',
 'descriptors',
 'dr',
 'events_dir',
 'evlist_dir',
 'l_max',
 'lib_path',
 'lmpData',
 'lmpDisp',
 'n_max',
 'nconf',
 'perAtomData',
 'positions',
 'r_cut',
 'scale',
 'shape',
 'species',
 'verbose']